In [ ]:
import sys
import os
import subprocess
import utils
import openai
from dotenv import load_dotenv
from pydantic import BaseModel
import json

In [ ]:
load_dotenv()

OPENAI_CLIENT = openai.Client()

In [ ]:
class DecompiledCode(BaseModel):
    code: str
    explanation: str

def decompile_with_gpt(disassembled_code):
    messages = [
        {"role": "system", "content": "You are an expert in assembly code, and you are responsible for decompling into C code the assembly code provided."},
        {"role": "user", "content": "You are given the following assembly code:"},
        {"role": "user", "content": disassembled_code},
        {"role": "user", "content": "Please provide the equivalent C code."},
    ]

    response = OPENAI_CLIENT.beta.chat.completions.parse(
        model='gpt-4o-mini',
        messages=messages,
        response_format=DecompiledCode,
        n=1,
        temperature=0.0,
    )

    response = json.loads(response.choices[0].message.content)

    decompiled_path = os.path.join(utils.WORKSPACE_DIR, "decompiled_code.c")
    with open(decompiled_path, "w") as f:
        f.write(response["code"])

    return response

In [ ]:
def process_file(input_path, function_name):
    # Copy the C code or binary to the workspace for reference
    try:
        disassembled_code = utils.disassemble(input_path, function_name)
        
        # Decompile with LLM4Decompile
        print("\nDecompiling...")
        response = decompile_with_gpt(disassembled_code)
        decompiled_code = response["code"]
        
        print("\nDecompiled Code:")
        print(decompiled_code)
    
    except subprocess.CalledProcessError as e:
        print(f"Error during processing: {e}")

In [ ]:
c_code_path = "c_code/fibonacci.c"
function_name = "func0"
process_file(c_code_path, function_name)